In [23]:
# Callback - 주기적으로 최적의 모델 자동 저정

In [24]:
# 패션 MNIST - 훈련 세트 & 테스트 세트

In [25]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [26]:
# 모델 생성 함수 - 층 추가가 가능

In [27]:
# 입력, 출력 이외의 층 추가가 없을 수도 있어서 default 값 = None
def model_fn (a_layer = None) :
    
    model = keras.Sequential()
    
    # 입력층
    model.add(keras.layers.Input(shape=(28,28)))

    # 평탄화 (2차원 -> 1차원)
    model.add(keras.layers.Flatten())

    # 은닉층 (밀집층)
    model.add(keras.layers.Dense(100, activation = 'relu'))

    # a_layer가 있을 경우 중간 중간 추가 가능하도록
    if a_layer :

        model.add(a_layer)
    
    # 출력층
    model.add(keras.layers.Dense(10, activation = 'softmax'))

    return model

In [28]:
# 30% 비율의 밀집층의 출력을 0으로 랜덤하게 변경
model = model_fn(keras.layers.Dropout(0.3))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# 데이터 전처리

In [30]:
train_scaled = train_input / 255
test_scaled = test_input / 255

In [31]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [32]:
# 검증 세트 정확도가 떨어지기 직전 시점에 모델을 파일로 저장
# 과대 적합 직전 시점에 모델을 파일로 저장

In [36]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.keras', save_best_only = True)

# 과대 적합 되기 직전까지만 훈련하고 종료 = 조기 종료
# 검증 세트의 정확도가 2회 이상 올라가지 않으면 조기 종료
# restore_best_weights = 가장 최적의 가중치로 복구
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 2, restore_best_weights=True)

model.fit(train_scaled, train_target, epochs = 20, validation_data = (test_scaled, test_target), callbacks = [checkpoint_cb, early_stopping_cb])

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8807 - loss: 0.3141 - val_accuracy: 0.8742 - val_loss: 0.3437
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8840 - loss: 0.3064 - val_accuracy: 0.8768 - val_loss: 0.3458
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8846 - loss: 0.3040 - val_accuracy: 0.8782 - val_loss: 0.3512


In [34]:
model = keras.models.load_model('best-model.keras')

In [35]:
model.evaluate(test_scaled, test_target)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.8797 - loss: 0.3381


[0.3431837558746338, 0.8791000247001648]

In [38]:
# 조기 종료 시점
early_stopping_cb.stopped_epoch

2